## Imports if don't acquire any of these package pip install it

In [10]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

### Adding the connection parameters and connect to mysql database


In [11]:
# Define your connection parameters
connection_params = {
    'host': 'localhost', # e.g., 'localhost' or '127.0.0.1'
    'database': 'platformdb1', # Name of your database
    'user': 'root', # Your MySQL username
    'password': '0000', # Your MySQL password
}

df = pd.read_excel('output2.xlsx',engine= 'openpyxl')
# convert the date type to a format to fit the mysql format
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y')

### Insert the data into the Tables dates,Brand,Category,Reigon, SKU and siZeBracket

In [20]:
try:
    # Establish the connection
    connection = mysql.connector.connect(**connection_params)
    
    if connection.is_connected():

        for table, column in [
            ('Brand', 'brand'),
            ('Segment', 'Segment'),
            ('Reigon', 'Reigon'),
            ('SKU', 'sku'),
            ('SizeBracket','size_bracket')
        ]:
            cursor = connection.cursor()
            unique_values = df[column].drop_duplicates().dropna().tolist() 

            data_to_insert = [(v,) for v in unique_values]

            if column == 'date': 
                query = f"INSERT INTO {table} ({column}) VALUES (%s)"
            else : 
                query = f"INSERT INTO {table} ({column}_name) VALUES (%s)"

            cursor.executemany(query, data_to_insert)
            connection.commit()

            print(f"successfully ingested into the {table}.")
    

except Error as e:
    print("Error while connecting to MySQL", e)


successfully ingested into the Brand.
successfully ingested into the Segment.
successfully ingested into the Reigon.
successfully ingested into the SKU.
successfully ingested into the SizeBracket.


### Adding The data into the rawdata table row by row from the excel file with matching id from other tables like SKU

It will take sometime 1:2 mins maybe

In [21]:
try:

    connection = mysql.connector.connect(**connection_params)

    cursor = connection.cursor()
    if connection.is_connected():

        category_id = size_bracket_id = sku_id = reigon_id = None

        # Now, insert data into the fact_sales table
        for index, row in df.iterrows():

            
            cursor.execute("SELECT brand_id FROM Brand WHERE brand_name = %s", (row['brand'],))
            brand_id = cursor.fetchone()[0]
            
            if pd.notna(row['Segment']) :
                cursor.execute("SELECT segment_id FROM Segment WHERE Segment_name = %s", (row['Segment'],))
                Segment_id = cursor.fetchone()[0]
            
            if pd.notna(row['size_bracket']) :
                cursor.execute("SELECT size_bracket_id FROM sizebracket WHERE size_bracket_name = %s", (row['size_bracket'],))
                size_bracket_id = cursor.fetchone()[0]
            
            if pd.notna(row['Reigon']) :
                cursor.execute("SELECT Reigon_id FROM Reigon WHERE Reigon_name = %s", (row['Reigon'],))
                reigon_id = cursor.fetchone()[0]

            if pd.notna(row['sku']) :
                cursor.execute("SELECT sku_id FROM SKU WHERE sku_name = %s", (row['sku'],))
                sku_id = cursor.fetchone()[0]
                
            # Prepare the dynamic SQL query and parameters
            columns = ['brand_id']
            values = [brand_id]
            

        
            if Segment_id is not None:
                columns.append('Segment_id')
                values.append(Segment_id)
            
            if size_bracket_id is not None:
                columns.append('size_bracket_id')
                values.append(size_bracket_id)
            
            if sku_id is not None:
                columns.append('sku_id')
                values.append(sku_id)

            if reigon_id is not None:
                columns.append('Reigon_id')
                values.append(reigon_id)

            columns.append('date')
            values.append(row['date'])

            
            if pd.notna(row['Sales (KGS) (000)']):
                columns.append('volume_sales')
                values.append(row['Sales (KGS) (000)'])

            if pd.notna(row['Sales (KGS) (000)']):
                columns.append('value_sales')
                values.append(row['Sales Value (000)'])
            
            if pd.notna(row['Weighted Distribution - Reach']):
                columns.append('WOD')
                values.append(row['Weighted Distribution - Reach'])

            # Build the final query
            insert_query = f"INSERT INTO RawData ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"
            
            # Execute the dynamic query
            cursor.execute(insert_query, tuple(values))

        # Commit the transaction
        connection.commit()
        print("Data inserted successfully into RawData table.")

    connection.close()
    
except Error as e:
    print("Error while connecting to MySQL", e)

Data inserted successfully into RawData table.


### Adding the data from to poromoted table with raw id 

In [6]:
connection = mysql.connector.connect(**connection_params)

cursor = connection.cursor()


# df.reset_index(inplace= True)
query = f"INSERT INTO Promotions (rawdata_id,Status) VALUES ({'%s, %s'})"

cursor.execute("SELECT Id FROM rawdata")
valid_rawdata_ids = list(set(row[0] for row in cursor.fetchall()))

df['index'] = valid_rawdata_ids

df['Promoted'].fillna(False, inplace=True) 
data_to_insert = df[['index', 'Promoted']].values.tolist()


cursor.executemany(query, data_to_insert)

# Commit the transaction
connection.commit()

connection.close()

### Drop all Table data
`don't use it unless there is an error and you want remove all data and start all over`

In [19]:
# connection = mysql.connector.connect(**connection_params)
# cursor = connection.cursor()

# # Disable foreign key checks
# cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")

# # Get all table names
# cursor.execute("SHOW TABLES;")
# tables = cursor.fetchall()

# # Truncate each table
# for table in tables:
#     cursor.execute(f"TRUNCATE TABLE {table[0]};")

# # Re-enable foreign key checks
# cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")

# # Close the connection
# connection.close()